In [3]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer
from qiskit.visualization import plot_histogram
backend = Aer.get_backend("unitary_simulator")

## $Z$ Gate

\begin{equation}
Z =
\begin{bmatrix}
1 & \\
  & -1 
\end{bmatrix}
\\ \\
HXH = Z
\end{equation}

In [4]:
z = QuantumCircuit(1);
z.h(0);
z.x(0);
z.h(0);

z.draw('mpl')

In [5]:
r = execute(z, backend=backend).result()
print(r.get_unitary(decimals=1))

[[ 1.-0.j -0.-0.j]
 [-0.+0.j -1.+0.j]]


## $CZ$ Gate

\begin{equation}
CZ =
\begin{bmatrix}
I &  \\
  & Z
\end{bmatrix}
\\ \\
(I\otimes H)CX(I \otimes H) = CZ
\end{equation}

In [35]:
cz = QuantumCircuit(2);
cz.h(0);
cz.cx(1,0);
cz.h(0);

cz.draw('mpl')

In [36]:
r = execute(cz, backend=backend).result()
print(r.get_unitary(decimals=1))

[[ 1.-0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j -0.-0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -1.+0.j]]


## $U_{10}$ (first reflection for s = 10 ?)

We want to implement: 
\begin{equation}
U_{10} =
\begin{bmatrix}
1 &  &  & \\
  & 1 & & \\
  &   & -1 & \\
  &   &   & 1 \\
\end{bmatrix}
\end{equation}



Idea: $U_{10} = (I \otimes X)\ CZ\  (I \otimes X)$


In [37]:
u_10 = QuantumCircuit(2, name="U_10")
u_10.x(0);
u_10.cz(1,0)
u_10.x(0);
u_10.draw('mpl')

## Verify $U_{10}$ is as expected

In [38]:
r = execute(u_10, backend=backend).result()
print(r.get_unitary(decimals=1))

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]


## $U_S$ (second reflection)


\begin{equation}
2|00\rangle\langle00| - I = 
\begin{bmatrix}
1 &  &  &  \\
 & -1 &  &  \\
 &  & -1 &  \\
 &  &  & -1 \\
\end{bmatrix}
\end{equation}

Idea: 
\begin{equation}
\cdots = 
\begin{bmatrix}
Z &  \\
  & -I  
\end{bmatrix}
= \cdots
\begin{bmatrix}
 & I \\
I  &   
\end{bmatrix}
\begin{bmatrix}
I &  \\
  & Z  
\end{bmatrix}
\begin{bmatrix}
 & I \\
I  &   
\end{bmatrix}
\end{equation}

In [50]:
u_s = QuantumCircuit(2,name="U_s")

u_s.h(0);
u_s.h(1);

u_s.x(1);
u_s.cz(1,0);
u_s.x(1);

u_s.z(1);

u_s.h(0);
u_s.h(1);

u_s.draw()

In [52]:
r = execute(u_s, backend=backend).result()
print(r.get_unitary(decimals=1))

[[-0.5+0.j  0.5-0.j  0.5-0.j  0.5-0.j]
 [ 0.5-0.j -0.5+0.j  0.5-0.j  0.5-0.j]
 [ 0.5-0.j  0.5-0.j -0.5+0.j  0.5-0.j]
 [ 0.5+0.j  0.5-0.j  0.5-0.j -0.5+0.j]]


## Grover $n=2$ with $S = \{10\}$

In [58]:
main = QuantumCircuit(2,2)

main.h(0);
main.h(1);

main.append(u_10.to_instruction(), [0,1]);

main.append(u_s.to_instruction(), [0,1]);

main.measure([0,1],[0,1]);

main.draw('mpl')

In [59]:
r = execute(main, backend=Aer.get_backend("qasm_simulator")).result()
r.get_counts()

{'10': 1024}